# Analísis de sentimientos

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import (
    TextAnalyticsClient,
    RecognizeEntitiesAction,
    AnalyzeSentimentAction,
)

credential = AzureKeyCredential("505c3ba3a57a4b749a5e5a96a589d39a")
endpoint="https://eastus2.api.cognitive.microsoft.com/"

text_analytics_client = TextAnalyticsClient(endpoint, credential)

In [ ]:
documents = [
    "La universidad Distrital tiene más presupueto que la Universidad Javeriana.",
    "La pandemia no solo trajo cosas malas, hemos aprendido mucho de ello",
    "Odio la gente gomela en los restaurantes",
    'Mañana sadré a comer con mi novia',
    'Hoy tengo clase de AI en la novhe, qué emoción',
    'mi perro Tobias debe lavarse los dientes'
]

response = text_analytics_client.analyze_sentiment(documents, language="es")
result = [doc for doc in response if not doc.is_error]

In [ ]:
for doc in result:
    print("Overall sentiment: {}".format(doc.sentiment))
    print("Scores: positive={}; neutral={}; negative={} \n".format(
        doc.confidence_scores.positive,
        doc.confidence_scores.neutral,
        doc.confidence_scores.negative,
    ))

# Reconocer entidades
Recognition_entities reconoce y categoriza entidades en su texto de entrada como personas, lugares, organizaciones, fecha / hora, cantidades, porcentajes, monedas y más.

In [ ]:
response = text_analytics_client.recognize_entities(documents, language="es")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    for entity in doc.entities:
        print("Entity: {}".format(entity.text))
        print("...Category: {}".format(entity.category))
        print("...Confidence Score: {}".format(entity.confidence_score))
        print("...Offset: {}".format(entity.offset))

# Reconocer entidades vinculadas
Recognition_linked_entities reconoce y elimina la ambigüedad de la identidad de cada entidad que se encuentra en su texto de entrada (por ejemplo, determina si una aparición de la palabra Marte se refiere al planeta o al dios romano de la guerra). Las entidades reconocidas se asocian con URL a una base de conocimientos conocida, como Wikipedia.


In [ ]:
response = text_analytics_client.recognize_linked_entities(documents, language="es")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    for entity in doc.entities:
        print("Entity: {}".format(entity.name))
        print("...URL: {}".format(entity.url))
        print("...Data Source: {}".format(entity.data_source))
        print("...Entity matches:")
        for match in entity.matches:
            print("......Entity match text: {}".format(match.text))
            print("......Confidence Score: {}".format(match.confidence_score))
            print("......Offset: {}".format(match.offset))

# Reconocer entidades PII
Recognition_pii_entities reconoce y categoriza entidades de información de identificación personal (PII) en su texto de entrada, como números de seguro social, información de cuentas bancarias, números de tarjetas de crédito y más.

In [ ]:
response = text_analytics_client.recognize_pii_entities(documents, language="es")
result = [doc for doc in response if not doc.is_error]

for idx, doc in enumerate(result):
    print("Document text: {}".format(documents[idx]))
    print("Redacted document text: {}".format(doc.redacted_text))
    for entity in doc.entities:
        print("...Entity: {}".format(entity.text))
        print("......Category: {}".format(entity.category))
        print("......Confidence Score: {}".format(entity.confidence_score))
        print("......Offset: {}".format(entity.offset))

# Extraer frases clave
extract_key_phrases determina los principales puntos de conversación en su texto de entrada. Por ejemplo, para el texto de entrada "La comida estaba deliciosa y había un personal maravilloso", la API devuelve: "comida" y "personal maravilloso".

In [ ]:
response = text_analytics_client.extract_key_phrases(documents, language="en")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    print(doc.key_phrases)

# Detectar idioma
detect_language determina el idioma de su texto de entrada, incluida la puntuación de confianza del idioma predicho.

In [ ]:
response = text_analytics_client.detect_language(documents)
result = [doc for doc in response if not doc.is_error]

for doc in result:
    print("Language detected: {}".format(doc.primary_language.name))
    print("ISO6391 name: {}".format(doc.primary_language.iso6391_name))
    print("Confidence score: {}\n".format(doc.primary_language.confidence_score))

# Análisis multiple

In [ ]:
poller = text_analytics_client.begin_analyze_actions(
    documents,
    display_name="Sample Text Analysis",
    actions=[
        RecognizeEntitiesAction(),
        AnalyzeSentimentAction()
    ]
)

# returns multiple actions results in the same order as the inputted actions
document_results = poller.result()
for doc, action_results in zip(documents, document_results):
    recognize_entities_result, analyze_sentiment_result = action_results
    print("\nDocument text: {}".format(doc))
    print("...Results of Recognize Entities Action:")
    if recognize_entities_result.is_error:
        print("......Is an error with code '{}' and message '{}'".format(
            recognize_entities_result.code, recognize_entities_result.message
        ))
    else:
        for entity in recognize_entities_result.entities:
            print("......Entity: {}".format(entity.text))
            print(".........Category: {}".format(entity.category))
            print(".........Confidence Score: {}".format(entity.confidence_score))
            print(".........Offset: {}".format(entity.offset))

    print("...Results of Analyze Sentiment action:")
    if analyze_sentiment_result.is_error:
        print("......Is an error with code '{}' and message '{}'".format(
            analyze_sentiment_result.code, analyze_sentiment_result.message
        ))
    else:
        print("......Overall sentiment: {}".format(analyze_sentiment_result.sentiment))
        print("......Scores: positive={}; neutral={}; negative={} \n".format(
            analyze_sentiment_result.confidence_scores.positive,
            analyze_sentiment_result.confidence_scores.neutral,
            analyze_sentiment_result.confidence_scores.negative,
        ))
    print("------------------------------------------")